### 必要なパッケージの読み込み

In [ ]:
import random
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

### セル空間（環境）のプログラム

In [ ]:
class CellSpace:
  def __init__(self, width, height):
    self.width = width
    self.height = height
    self.cells = [[None for _ in range(height)] for _ in range(width)]
    self.agent_list = []

  def register_agent(self, agent):
    self.agent_list.append(agent)
  
  def is_valid_position(self, x, y):
    ret = False
    if 0 <= x and x < self.width and 0 <= y and y < self.height:
      ret = True
    return ret

  def set_at(self, x, y, object):
    if self.is_valid_position(x, y):
      self.cells[x][y] = object

  def get_at(self, x, y):
    ret = None
    if self.is_valid_position(x, y):
      ret = self.cells[x][y]
    return ret

  def update(self):
    for agent in self.agent_list:
      agent.update()
  
  def draw(self, axis):
    colors = ["Red", "Blue", "Black", "Yellow", "Green", "Magenta", "Cyan"]
    kinds = []
    x_lists = {}
    y_lists = {}
    for agent in self.agent_list:
      if agent.kind() not in kinds:
        kinds.append(agent.kind())
        x_lists[agent.kind()] = []
        y_lists[agent.kind()] = []
      x_lists[agent.kind()].append(agent.x)
      y_lists[agent.kind()].append(agent.y)
    
    imgs = []
    for i, k in enumerate(kinds):
      imgs.append(axis.scatter(x_lists[k], y_lists[k], color=colors[i]))
    return imgs


### エージェントのベースクラス

In [ ]:
class Agent:
  def __init__(self, space, x, y):
    self.space = space
    self.x = 0
    self.y = 0
    self.set_position(x, y)

  def set_position(self, x, y):
    self.space.set_at(self.x, self.y, None)
    self.x = x
    self.y = y
    self.space.set_at(self.x, self.y, self)

  def get_neighbors(self):
    empty_cells = []
    neighbors = []

    dx_list = [-1, -1, -1, 0, 0, 1, 1, 1]
    dy_list = [-1, 0, 1, -1, 1, -1, 0, 1]
    for dx, dy in zip(dx_list, dy_list):
      nx = self.x + dx
      ny = self.y + dy
      if self.space.is_valid_position(nx, ny):
        if self.space.get_at(nx, ny) is None:
          empty_cells.append((nx, ny))
        else:
          neighbors.append(self.space.get_at(nx, ny))
    return neighbors, empty_cells

  def count_neighbor_kinds(self):
    neighbors, _ = self.get_neighbors()
    same_num = 0
    other_num = 0
    for n in neighbors:
      if n.kind() == self.kind():
        same_num += 1
      else:
        other_num += 1
    return same_num, other_num

  def move(self):
    _, empty_list = self.get_neighbors()
    if 0 < len(empty_list):
      new_x, new_y = empty_list[random.randrange(len(empty_list))]
      self.set_position(new_x, new_y)

  def update(self):
    same_num, other_num = self.count_neighbor_kinds()
    if self.need_to_move(same_num, other_num):
      self.move()

  def kind(self):
    pass

  def need_to_move(self, same_num, other_num):
    pass


### 個別のエージェントのプログラム（許容水準）
- AgentA: 近隣の同種の数が2以上
- AgentB: 近隣の同種の数が近隣の駒数の1/3より多い

In [ ]:
class AgentA(Agent):
  def kind(self):
    return "A"

  def need_to_move(self, same_num, other_num):
    yn = False
    if same_num < 2:
      yn = True
    return yn


In [ ]:
class AgentB(Agent):
  def kind(self):
    return "B"
  
  def need_to_move(self, same_num, other_num):
    yn = False
    if 3 * same_num <= (same_num + other_num):
      yn = True
    return yn    

### 設定
- WIDTH, HEIGHT: セル空間の大きさ
- AGENTS: エージェントの種類と数

In [ ]:
WIDTH = 100
HEIGHT = 100
AGENTS = {AgentA:1000, AgentB:1000}

### シミュレーション

In [ ]:
def img_update(i, space, ax):
  ax.cla()
  space.update()
  return space.draw(ax)

space = CellSpace(WIDTH, HEIGHT)
pos_list = [(x,y) for x in range(WIDTH) for y in range(HEIGHT)]
for kind in AGENTS.keys():
  for _ in range(AGENTS[kind]):
    pos = pos_list[random.randrange(len(pos_list))]
    a = kind(space, pos[0], pos[1])
    space.register_agent(a)
    pos_list.remove(pos)

fig, ax = plt.subplots()

ani = animation.FuncAnimation(fig, img_update, fargs=(space, ax), interval=100, frames=500)
HTML(ani.to_jshtml())